In [3]:
%pip install langchain langchain_openai PyPDF2 faiss-cpu chromadb google-api-core google-api-python-client google-auth google-auth-httplib2 googleapis-common-protos html2text --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv

EMBEDDINGS_DEPLOYMENT_NAME = "text-embedding-ada-002"
EMBEDDINGS_API_VERSION = "2023-08-01-preview"

DEPLOYMENT_NAME="gpt-35-turbo"
MODEL_NAME = "gpt-35-turbo"
API_VERSION="2023-12-01-preview"

load_dotenv()

True

In [5]:
# Vectorstore
vectorstore = Chroma(
    embedding_function=AzureOpenAIEmbeddings(
        azure_deployment=EMBEDDINGS_DEPLOYMENT_NAME,
        openai_api_version=EMBEDDINGS_API_VERSION), 
        persist_directory="./chroma_db_oai"
)

# LLM
llm = AzureChatOpenAI(
    azure_deployment=DEPLOYMENT_NAME, 
    model_name=MODEL_NAME, 
    openai_api_version=API_VERSION,
    temperature=0)

# Search
search = GoogleSearchAPIWrapper()

In [6]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

In [8]:
# import logging
# logging.basicConfig()
# logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
from langchain.chains import RetrievalQAWithSourcesChain

user_input = "Does Azure ExpressRoute support multiple circuits in same peering location from same ISP?"
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever, return_source_documents=True
)

result = qa_chain.invoke({"question": user_input})

print(result)


INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'Does Azure ExpressRoute support multiple circuits in same peering location from same ISP?', 'text': ['1. Can Azure ExpressRoute accommodate multiple circuits from the same ISP at the same peering location?\n', '2. Is it possible to have multiple circuits from the same ISP in the same peering location with Azure ExpressRoute?\n', '3. Does Azure ExpressRoute allow for multiple circuits from the same ISP to be used in the same peering location?']}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. Can Azure ExpressRoute accommodate multiple circuits from the same ISP at the same peering location?\n', '2. Is it possible to have multiple circuits from the same ISP in the same peering location with Azure ExpressRoute?\n', '3. Does Azure ExpressRoute allow for multiple circuits from the same ISP to be us

{'question': 'Does Azure ExpressRoute support multiple circuits in same peering location from same ISP?', 'answer': 'Yes. You can have multiple ExpressRoute circuits with the same or different service providers. If the metro has multiple ExpressRoute peering locations and the circuits are created at different peering locations, you can link them to the same virtual network. If the circuits are created at the same peering location, you can link up to four circuits to the same virtual network.\n', 'sources': 'https://learn.microsoft.com/en-us/azure/expressroute/expressroute-faqs', 'source_documents': [Document(page_content='### Can I have ExpressRoute circuits from different service providers?\n\nYes. You can have ExpressRoute circuits with many service providers. Each\nExpressRoute circuit is associated with one service provider only.\n\n### I see two ExpressRoute peering locations in the same metro, for example,\nSingapore and Singapore2. Which peering location should I choose to creat